# Music generation with Bach Chorales

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Packages and data
for annotation

In [2]:
!pip install music21;

### upload midi file


In [4]:
from google.colab import files
files.upload()

Saving kept_bach_midi.zip to kept_bach_midi.zip


{'kept_bach_midi.zip': b'PK\x03\x04\x14\x00\x00\x00\x00\x00uX\x8fO\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x00 \x00kept_bach_midi/UT\r\x00\x07^Y\xf6]^Y\xf6]^Y\xf6]ux\x0b\x00\x01\x04\xf5\x01\x00\x00\x04\x14\x00\x00\x00PK\x03\x04\x14\x00\x08\x00\x08\x00\xe0V\x8fO\x00\x00\x00\x00\x00\x00\x00\x00\x8e\x07\x00\x00\x1b\x00 \x00kept_bach_midi/042100b_.midUT\r\x00\x07TW\xf6]]W\xf6]TW\xf6]ux\x0b\x00\x01\x04\xf5\x01\x00\x00\x04\x14\x00\x00\x00\x9d\x93\xbbn\x13Q\x10\x86\xc7\x89\xc3\xbdF\x94)h\xa3\xf5e\xad\xc4\xb7\xc4\x978\xb6\xa3\xb8@v\x01]\x10\x04\x05\xa1\x18\xe1l\n:\xbc\xbb\xc5\x16)\x88\xdd\xecc\xe4B\xc73\xf0\x0c\xf0&\xcb\xfc\xe7\xacG\xc7YcY\xb8\xfav\xe6\x9f\x99\xff\x9c9\xee\rN\xdf\x13\xd1\x03J\xd1F\x9az\x83\xd1\'\xfezI\xd1`\x83\xd4/z\x9dN\xaf\xbdxD\xd1\x9b5\x96D\xaf\xd6\x9f~\xf9\xe1\xfeIEV\xacM\xfd\xa4\xe8\xf1\xf3\xfe\xd9\xdb\x91\xd3\xbb8\xff\xf8n\xb3\xff\xf9\x83\xd3\xff:tN7\xb3\x14\xad?\xeb\x0e\xcf\x9d\xd1\xc5\xd9\xc9\xd0\xe1\xef\xeb\x87\'t\xfd\xa4\x16\xd2\xf7\xd6q@\xdfZ\x04\xf05\x

In [5]:
!unzip kept_bach_midi.zip;

Archive:  kept_bach_midi.zip
   creating: kept_bach_midi/
  inflating: kept_bach_midi/042100b_.mid  
  inflating: __MACOSX/kept_bach_midi/._042100b_.mid  
  inflating: kept_bach_midi/043100b_.mid  
  inflating: __MACOSX/kept_bach_midi/._043100b_.mid  
  inflating: kept_bach_midi/027400b_.mid  
  inflating: __MACOSX/kept_bach_midi/._027400b_.mid  
  inflating: kept_bach_midi/026400b_.mid  
  inflating: __MACOSX/kept_bach_midi/._026400b_.mid  
  inflating: kept_bach_midi/006206b_.mid  
  inflating: __MACOSX/kept_bach_midi/._006206b_.mid  
  inflating: kept_bach_midi/007206b_.mid  
  inflating: __MACOSX/kept_bach_midi/._007206b_.mid  
  inflating: kept_bach_midi/024432b_.mid  
  inflating: __MACOSX/kept_bach_midi/._024432b_.mid  
  inflating: kept_bach_midi/004705b_.mid  
  inflating: __MACOSX/kept_bach_midi/._004705b_.mid  
  inflating: kept_bach_midi/016806b_.mid  
  inflating: __MACOSX/kept_bach_midi/._016806b_.mid  
  inflating: kept_bach_midi/012506b_.mid  
  inflating: __MACOSX/kept

## Processing data
load them into **music21**

In [6]:
# Importing dependencies
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


**music21** represents music, two different kind of elements:
- **Notes**
- **Chords**

Also note the offset of each element. This is the time when the note or chord must be played.

In [7]:
file = "kept_bach_midi/000306b_.mid"
midi = converter.parse(file)
notes_to_parse = midi.flat.notes
for element in notes_to_parse[:10]:
  print(element, element.offset)

<music21.note.Note A> 3.0
<music21.note.Note C#> 3.0
<music21.note.Note A> 3.0
<music21.note.Note A> 3.0
<music21.note.Note D> 3.5
<music21.note.Note B> 3.5
<music21.note.Note A> 4.0
<music21.note.Note E> 4.0
<music21.note.Note C#> 4.0
<music21.note.Note A> 4.0


process all MIDI files obtaining data from each note of chord.

In [8]:
notes = []
notes_test = []
for i,file in enumerate(glob.glob("kept_bach_midi/*.mid")):
  # if i <= 305:
    midi = converter.parse(file)
    print('\r', 'Parsing file ', i, " ",file, end='')
    notes_to_parse = None
    try: # file has instrument parts
      s2 = instrument.partitionByInstrument(midi)
      notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))
'''
  else:
    midi = converter.parse(file)
    print('\r', 'Parsing file ', i, " ",file, end='')
    notes_to_parse = None
    try: # file has instrument parts
      s2 = instrument.partitionByInstrument(midi)
      notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
      if isinstance(element, note.Note):
        notes_test.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes_test.append('.'.join(str(n) for n in element.normalOrder))
        '''

with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

'''
with open('notes_test', 'wb') as filepath:
  pickle.dump(notes_test, filepath)
  '''

 Parsing file  383   kept_bach_midi/008606b_.mid

"\nwith open('notes_test', 'wb') as filepath:\n  pickle.dump(notes_test, filepath)\n  "

We obtain the number of different notes in our dataset, because this will be the **number of possible output classes**  of our model.

In [9]:
# Count different possible outputs
n_vocab = (len(set(notes)))
n_vocab

88

In [10]:
n_vocab_test = (len(set(notes_test)))
n_vocab_test

0

In [11]:
# number of total notes in test set
len(notes_test)

0

In [12]:
# number of total notes in train set
len(notes)

110471

- map each pitch or chord to an integer
- create pairs of input sequences and its corresponding output note

can try different **sequence_length** to obtain different results. use a sequence_length of 100 for now.

The network will made its prediction of the next note (or chord), based on the previous `sequence_length` many notes (or chords). 

In [0]:
sequence_length = 150
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length] # Size sequence_length
  sequence_out = notes[i + sequence_length]  # Size 1
  # Map pitches of sequence_in to integers
  network_input.append([note_to_int[char] for char in sequence_in])
  # Map integer of sequence_out to an integer
  network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [14]:
network_input.shape

(110321, 150, 1)

## Creating model

create a network with 9 layers (3 of them **LSTM layers**).
For regularization, add 2 Dropout layers

In [0]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.1))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.1))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [16]:
model = create_network(network_input,n_vocab)
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 150, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 150, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    

can use previously trained weights to continue the training where we left off, we should load them into the model. 

This is very useful in Google Colaboratory, that usually kills the virtual machine that is executing the Jupyter notework after a certime amount of time. If this happens to you, you should have to look for the last weights file in your configured Drive account and use it to train the network.

In [0]:
# In case we want to use previously trained weights
weights = ""
if(len(weights)>0): model.load_weights(weights)

ModelCheckpoint will save weights in a file after each epoch.

This way, can resume start execution where we left if the training stops.

You can train as many epochs as you want.




In [0]:
filepath = "/content/drive/My Drive/bach_full_weights/{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,
                             save_best_only=True,mode='min')

callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=77, batch_size=100,
          callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/77





110321/110321 [==============================] - 646s 6ms/step - loss: 3.4489
Epoch 2/77
110321/110321 [==============================] - 645s 6ms/step - loss: 3.4227
Epoch 3/77
110321/110321 [==============================] - 647s 6ms/step - loss: 3.4204
Epoch 4/77
110321/110321 [==============================] - 646s 6ms/step - loss: 3.4203
Epoch 5/77
110321/110321 [==============================] - 649s 6ms/step - loss: 3.4201
Epoch 6/77
110321/110321 [==============================] - 648s 6ms/step - loss: 3.4200
Epoch 7/77
110321/110321 [==============================] - 648s 6ms/step - loss: 3.4202
Epoch 8/77
110321/110321 [==============================] - 653s 6ms/step - loss: 3.4202
Epoch 9/77
110321/110321 [==============================] - 653s 6ms/step - loss: 3.4205
Epoch 10/77
110321/110321 [==============================] - 655s 6ms/step - loss: 3.4203
Epoch 1

## Music generation

There can be generated songs that sound awful, but try to execute the generation
process several times to get interesting results.

In [18]:
# In case we want to use other previously trained weights

weights = "/content/drive/My Drive/bach_full_weights/final.h5"

##weights = "/content/drive/My Drive/final-weights.h5"
if(len(weights)>0): model.load_weights(weights)

In [0]:
# Generate network input again
network_input = []
output = []
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

The workflow now is:


1.   Pick a **seed sequence** randomly from your list of inputs (*pattern* variable)
2.   Pass it as input for your model to generate a new element (note or chord)
3.   Add the new element to your final song and to your *pattern* list
4.   Remove the first item from *pattern*
5.   Go to step 2




In [20]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes, but for our case let's start with 200
for i,note_index in enumerate(range(200)):
  prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
  prediction_input = prediction_input / float(n_vocab)
  prediction = model.predict(prediction_input, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  prediction_output.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

 Predicted  199   D4

create a MIDI file from the predictions.

In [21]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'